This notebook walks you through the process of uploading a new project to the annotation service. First make sure that the website is actually runnung and that the data in `../data/config.json` is correct. Note that you most likely want to run this notebook on the same platform as the webserver is running to reduce errors caused by dropped http-requests

In [ ]:
import json
import os
import glob

from api_wrapper.main import init
from api_wrapper.projects import *
from api_wrapper.img_batches import *
from api_wrapper.example_image import *
from api_wrapper.points_and_joints import *

init("../data/config.json")

In [ ]:
project_folder = "./dummy_data/Rhinos"
project_name = "Rhinos"

## Folder structure

We expect the `project_folder` to have the follwing structure/subdirectories:

- `/meta`
    - `*.json`
    - `*.jpg or *.png`
    - `*.jpg or *.png`
    - `...`
   
- `/*`
    - `/*`
        - `*.jpg or *.png`
        - `...`
    - `...`
   
- `...`

Here `*` indicates any abitrary name. Have a look at `/api_handling/dummy_data/Rhinos` for an example hot the files should look like.

In `/meta` the unique JSON file holds the project config data. This is data about which points and joints exist, as well as data for the example images (the image files in that directory.)
The other folders are for the specific batches. The names of the top level folders dont matter and are just for your organization. (E.g. you may choose to only have one.) The names of the second level folders are also gonna be the names of the corresponding batches. Lastly the image files in those secondary folders are the images for the specific batch.

## Creating the project

See `/api_handling/docs.md` for a full documentation of the used methods

Each method which interacts with the webserver returns an object with an error flag and the result. If no error occured everything is great. If an error occured and you fixed your code you can either

- Use the API methods to fix any damage and try again or
- Delete the project and run the code again

In most cases I recommend the second approach as it is less work and doesn't loose you anything. You delete a project by calling:

`delete_project(project_id)`

You can find the id for the project in the url when navigating to it on the website.

In [ ]:
project = create_project(project_name)
project

### Set Points & Joints

In [ ]:
json_files = glob.glob(os.path.join(project_folder, "meta", "*.json"))
if len(json_files) != 1:
    raise ValueError("There should be exactly one JSON file in the project folder.")
json_file_name = os.path.basename(json_files[0])

In [ ]:
with open(
    os.path.join(project_folder, "meta", json_file_name), 'r', encoding='utf-8'
) as file:
    project_data = json.load(file)
    
points = {}
for pt in project_data["points"]:
    points[pt] = create_point(project["id"], pt)
    
joints = []
for (x, y) in project_data["joints"]:
    joints.append(create_joint(points[x]["point_id"], points[y]["point_id"]))

points, joints

### Set example images

In [ ]:
i = 0
for img in project_data["images"]:
    i += 1
    ex_img = upload_example_image(project["id"],  os.path.join(project_folder, "meta", img["name"]), f"Image { i }")
    print(ex_img)
    
    dimensions = img["resolution"]
    img_points = img["points"]
    
    print(img_points)

    set_example_img_points(ex_img["id"], [{
            "point_id":      points[key]["point_id"],
            "widthPercent":  img_points[key][0]/dimensions[0] * 100,
            "heightPercent": img_points[key][1]/dimensions[1] * 100
        } for key in img_points.keys() if img_points[key][0]])

### Upload image batches

In [ ]:
for folder in os.listdir(project_folder):
    folder_path = os.path.join(project_folder, folder)
    
    if folder != "meta" and os.path.isdir(folder_path):
        # Iterate over batch folders within each folder
        for batch_folder in os.listdir(folder_path):
            batch_folder_path = os.path.join(folder_path, batch_folder)
            
            # Create an image batch for each batch folder
            img_batch = create_batch(project["id"], batch_folder)
            
            # Iterate over files in the batch folder
            for fp in os.listdir(batch_folder_path):
                absolute_fp = os.path.join(batch_folder_path, fp)
                
                # Upload each image to the image batch
                r = upload_image_to_img_batch(
                    img_batch["batch_id"],
                    absolute_fp
                )
                
                if (r["error"]):
                    print(r["value"])

## Misc

You probably want to make sure not commit the computes of this .ipynb file you do